In [32]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nlp
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [33]:
validation = pd.read_csv('./validation.csv')
train = pd.read_csv('./training.csv')
test = pd.read_csv('./test.csv')

In [75]:
train_text = train['text']
train_labels = train['label']

In [35]:
train_text[0], train_labels[0]

('i didnt feel humiliated', 0)

In [44]:
tokenizer = Tokenizer(num_words=10000, oov_token='<UNK>', split=' ', char_level=False)
tokenizer.fit_on_texts(train_text)

In [45]:
train_text[0]

'i didnt feel humiliated'

In [49]:
tokenizer.texts_to_sequences([train_text[0]])

[[2, 139, 3, 679]]

In [50]:
maxlen=50
def get_sequences(tokenizer, train_text):
    sequences = tokenizer.texts_to_sequences(train_text)
    padded = pad_sequences(sequences, truncating = 'post', padding='post', maxlen=maxlen)
    return padded

In [100]:
padded_train_seq = get_sequences(tokenizer, train_text)

In [102]:
padded_train_seq[0]

array([  2, 139,   3, 679,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=int32)

In [69]:
classes = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
class_to_index = dict((c,i) for i, c in enumerate(classes))
index_to_class = dict((v,k) for k, v in class_to_index.items())
names_to_ids = lambda labels: np.array([class_to_index.get(x) for x in labels])

In [70]:
print(classes)

['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']


In [71]:
class_to_index

{'sadness': 0, 'joy': 1, 'love': 2, 'anger': 3, 'fear': 4, 'surprise': 5}

In [74]:
index_to_class

{0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}

In [77]:
print(train_labels[0])

0


In [62]:
model = tf.keras.models.Sequential([
tf.keras.layers.Embedding(10000,16,input_length=maxlen),
tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20, return_sequences=True)),
tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20)),
tf.keras.layers.Dense(6, activation='softmax')
])
model.compile(
     loss='sparse_categorical_crossentropy',
     optimizer='adam',
     metrics=['accuracy']
)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 50, 16)            160000    
                                                                 
 bidirectional_4 (Bidirecti  (None, 50, 40)            5920      
 onal)                                                           
                                                                 
 bidirectional_5 (Bidirecti  (None, 40)                9760      
 onal)                                                           
                                                                 
 dense_2 (Dense)             (None, 6)                 246       
                                                                 
Total params: 175926 (687.21 KB)
Trainable params: 175926 (687.21 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [122]:
val_text = validation['text']
val_labels = validation['label']
val_seq = get_sequences(tokenizer, val_text)

In [126]:
h = model.fit(
     padded_train_seq, train_labels, batch_size = 100,
     validation_data=(val_seq, val_labels),
     epochs=20,
     callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=0.01, patience=5)]
)

Epoch 1/20
160/160 [==============================] - 3s 16ms/step - loss: 0.0376 - accuracy: 0.9894 - val_loss: 0.4483 - val_accuracy: 0.9030
Epoch 2/20
160/160 [==============================] - 3s 16ms/step - loss: 0.0342 - accuracy: 0.9909 - val_loss: 0.4588 - val_accuracy: 0.9010
Epoch 3/20
160/160 [==============================] - 3s 16ms/step - loss: 0.0368 - accuracy: 0.9901 - val_loss: 0.4635 - val_accuracy: 0.8980
Epoch 4/20
160/160 [==============================] - 3s 16ms/step - loss: 0.0332 - accuracy: 0.9902 - val_loss: 0.4692 - val_accuracy: 0.9015
Epoch 5/20
160/160 [==============================] - 3s 16ms/step - loss: 0.0307 - accuracy: 0.9911 - val_loss: 0.4842 - val_accuracy: 0.9000
Epoch 6/20
160/160 [==============================] - 3s 16ms/step - loss: 0.0288 - accuracy: 0.9916 - val_loss: 0.4887 - val_accuracy: 0.9005


In [127]:
test_text = test['text']
test_labels=test['label']
test_seq = get_sequences(tokenizer, test_text)

model.evaluate(test_seq, test_labels)

63/63 [==============================] - 0s 2ms/step - loss: 0.5074 - accuracy: 0.8825


[0.5073844790458679, 0.8824999928474426]

In [128]:
model.save("./emotion_model.h5")

/Users/henrytse/code/AppMath/appMath/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
